In [ ]:
import os
import torch
import torch.nn.functional as F
import pytorch_lightning as pl;
import importlib
import matplotlib.pyplot as plt
import numpy as np
import data
import utils
import sys
import importlib
import wandb
import copy
import pickle

from torch import nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from abc import ABC,abstractmethod
from torch.utils.data import DataLoader, Subset
from copy import deepcopy
from torch.special import logit
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from timeit import default_timer as timer

importlib.reload(data)
importlib.reload(utils)

pl.seed_everything(42)

debug=False

device=torch.device('mps') if (torch.backends.mps.is_available() and torch.backends.mps.is_built()) else torch.device('cpu')
print(device)


In [ ]:
#testing masked scl

import masking,data,utils
from data import IRAVENDataModule
importlib.reload(masking)
importlib.reload(data)
importlib.reload(utils)

from models.cnn_mlp import CNN_MLP

#setup
train=False
test=True
logging=False
load=False
model_ckpt=None
task_path='datasets/squares'
save_freq= 10000
batch_size=8
logit_init=2.5
split=(90,10,0)
device=torch.device('cpu')

#dataset setup
if 1:
    #task dataset
    path=task_path
    data_module=IRAVENDataModule(batch_size=batch_size,split=split)
    data_module.prepare_data()
    data_module.setup(root_dir=path)
    train_dataloader_task,test_dataloader_task=data_module.train_dataloader(),data_module.test_dataloader()
    x,y,*rest=next(iter(train_dataloader_task))

    #NOT task dataset
    path_='datasets/originals_masking'
    data_module_=IRAVENDataModule(batch_size=batch_size,split=split)
    data_module_.prepare_data()
    data_module_.setup(root_dir=path_)
    test_dataloader_not_task=data_module_.test_dataloader()

#setup model
if 1:

    cnn_mlp_kwargs={
    
        }


    model=CNN_MLP(**cnn_mlp_kwargs)

    if load:
    #load
        state_dict=torch.load(model_ckpt)
        model.load_state_dict(state_dict)
        
    model.eval() #for batch norm

#initialise masked model
if 1: 
    init_kwargs={
        'model':model,
        'train_dataloader':train_dataloader_task,
        'test_dataloader1':test_dataloader_task,
        'test_dataloader2':test_dataloader_not_task,
        'device':device,
        'savedir':'model_ckpts/FFN',
        'logit_init':2.5,
    }

    masked_cnn_mlp=masking.MaskedCNN_MLP(init_kwargs)

#train
if 1:
    train_kwargs={
        'alpha':1e-6,
        'tau':1,
        'lr':1e-3,
        'n_epochs':2 if test else 10,
        'n_batches':20 if test else 'full',
        'val_every_n_steps':5 if test else 10,
        'eval_every_n_steps':1e10,
        'n_val_batches':2 if test else 100,
        'n_eval_batches':2 if test else 100,
        'save_freq':save_freq,
        'logging':logging,

        }

    if train:
        masked_cnn_mlp.train(**train_kwargs)


In [ ]:
##sanity check
out_model=model(x)
out_masked_model=masked_cnn_mlp.forward(x)
print(F.mse_loss(out_model,out_masked_model).item())

out_conv=model.conv(x)
out_masked_conv=masked_cnn_mlp.MaskedConvModule(x)
print(F.mse_loss(out_conv,out_masked_conv).item())